In [ ]:
import torch

# Odszumianie obrazów


## Autoenkoder


## Variational Autoencoder

# Modele Generatywne

## Generative Adversarial Networks

## Dyfuzja

# Inpainting

## Dyfuzja + maskowanie (dopytać Asi)?

# Modele 3D